In [0]:
sc

Out[1]:

SparkContext 

 Spark UI 

 
 Version 
 v3.1.0 
 Master 
 local[8] 
 AppName 
 Databricks Shell

1. Dla zbioru recenzji Amazona wysupłaj te, które mają najwyższy rating (5.0) oraz zawierają słowo mouse.

In [0]:
path = "/databricks-datasets/amazon/data20K/part-*.gz.parquet"
rdd = spark.read.parquet(path).rdd

In [0]:
rdd_filtered = rdd.filter(lambda x: x.rating == 5)



In [0]:

rdd_filtered2 = rdd.filter(lambda x: ("mouse" in x.review) & (x.rating == 5))
rdd_filtered2.take(1)

Out[97]: [Row(rating=5.0, review="This mouse is amazing, I had owned a Razer Naga, and a R.A.T. 7 mouse. And I am not afraid to say that this mouse takes the cake, for $20 it looks like its worth $80. The only issue i had was that the seller sent me a wireless version, but I don't mind because it works just as well. Also where you see silver on the mouse in reality it is glossy black, and the mouse looks better IRL then in the pictures. I will add on to this review in a month or so to check in.")]

2. Twoim zadaniem będzie policzyć najpopularniejsze żądania HTTP ze zbioru dostępnego w domyślnych danych Databricks. Mamy na myśli logi Apache z folderu /databricks-datasets/sample_logs/.

Naszym punktem zainteresowań są tylko żądania dla requestów na endpointy o numerach od 300 do 399. Korzystając z poznanych tutaj transformacji (map, filter) oraz wyliczeń po ilości (countByValue), stwórz statystyki mówiące, które z tych endpointów są najbardziej popularne – z którymi z nich było najwięcej połączeń.

In [0]:
rdd2 = sc.wholeTextFiles("/databricks-datasets/sample_logs/part-*")

In [0]:
import re

rdd5 =(rdd2.map(lambda line: re.search('"*/endpoint_3.*"',line[1])))


sorted(
    rdd5.countByKey().items(),
    key=lambda el:el[1], reverse=True
)[:10]


Out[99]: [('/endpoint_322 HTTP/1.1"', 8),
 ('/endpoint_368 HTTP/1.1"', 5),
 ('/endpoint_381 HTTP/1.1"', 4),
 ('/endpoint_314 HTTP/1.1"', 4),
 ('/endpoint_356 HTTP/1.1"', 4),
 ('/endpoint_343 HTTP/1.1"', 4),
 ('/endpoint_304 HTTP/1.1"', 2),
 ('/endpoint_345 HTTP/1.1"', 1)]

Zajmijmy się zbiorem piosenek, który znajdziesz pod ścieżką /databricks-datasets/songs/data-001. Z kolekcji całych plików (wholeTextFiles) stwórz RDD, a następnie korzystając z mapowania, przekształć na RDD zawierającą nazwy plików i ich rozmiar.

In [0]:
%fs
ls /databricks-datasets/songs/

path,name,size
dbfs:/databricks-datasets/songs/README.md,README.md,1719
dbfs:/databricks-datasets/songs/data-001/,data-001/,0
dbfs:/databricks-datasets/songs/data-002/,data-002/,0


Po raz kolejny zajmijmy się zbiorem piosenek, który znajdziesz pod ścieżką /databricks-datasets/songs/data-001. Z kolekcji całych plików (wholeTextFiles) stwórz RDD, a następnie korzystając z mapowania, przekształć na RDD zawierającą nazwy plików i ich rozmiar.

In [0]:
rdd_files = sc.wholeTextFiles("/databricks-datasets/songs/data-001")

rdd_files.take(1)

Out[100]: [('dbfs:/databricks-datasets/songs/data-001/header.txt',
 'artist_id:string\nartist_latitude:double\nartist_longitude:double\nartist_location:string\nartist_name:string\nduration:double\nend_of_fade_in:double\nkey:int\nkey_confidence:double\nloudness:double\nrelease:string\nsong_hotnes:double\nsong_id:string\nstart_of_fade_out:double\ntempo:double\ntime_signature:double\ntime_signature_confidence:double\ntitle:string\nyear:double\npartial_sequence:int')]

In [0]:
rdd_map = rdd_files.map(lambda text: (text[0],  text[1]))

rdd_map.take(1)

Out[101]: [('dbfs:/databricks-datasets/songs/data-001/header.txt',
 'artist_id:string\nartist_latitude:double\nartist_longitude:double\nartist_location:string\nartist_name:string\nduration:double\nend_of_fade_in:double\nkey:int\nkey_confidence:double\nloudness:double\nrelease:string\nsong_hotnes:double\nsong_id:string\nstart_of_fade_out:double\ntempo:double\ntime_signature:double\ntime_signature_confidence:double\ntitle:string\nyear:double\npartial_sequence:int')]

In [0]:
rdd_map3 = rdd_files.map(lambda text: (text[0], len(text[1])))

rdd_map3.take(2)

Out[102]: [('dbfs:/databricks-datasets/songs/data-001/header.txt', 377),
 ('dbfs:/databricks-datasets/songs/data-001/part-00000', 52793)]

Podejrzyj zbiór adult poleceniem take. Z czego składa się każdy element? To tzw. Row z elementami oznaczonymi _c0, _c1 itd. Są to nienazwane kolumny (można oczywiście je nazwać przy pomocy API DataFrame). Dla nas najważniejsze będą:

_c0: wiek
_c5: stan cywilny
_c6: zawód

Na początek stwórz RDD, które będzie zawierać tylko takie osoby, które są rozwiedzione: _c5 zawiera słowo Divorced. Wypisz 10 pierwszych osób z tego zestawienia, posortowanych według wieku rosnąco.

Teraz dokonaj dalszej analizy. Weź pod uwagę zawód. Oblicz, jakim procentem dla tych wszystkich rozwiedzionych są osoby, które pracują na stanowiskach menedżerskich: właściwością _c6 jest Exec-managerial.

In [0]:
rdd = spark.read.csv("/databricks-datasets/adult/adult.data").rdd

In [0]:
rdd.take(1)

Out[104]: [Row(_c0='39', _c1=' State-gov', _c2=' 77516', _c3=' Bachelors', _c4=' 13', _c5=' Never-married', _c6=' Adm-clerical', _c7=' Not-in-family', _c8=' White', _c9=' Male', _c10=' 2174', _c11=' 0', _c12=' 40', _c13=' United-States', _c14=' <=50K')]

In [0]:
 #   _c0: wiek
 #   _c5: stan cywilny
 #   _c6: zawód

In [0]:
rdd2 = rdd.map(lambda x: [x[i] for i in [0,5,6]])
rdd2.take(5)

Out[106]: [['39', ' Never-married', ' Adm-clerical'],
 ['50', ' Married-civ-spouse', ' Exec-managerial'],
 ['38', ' Divorced', ' Handlers-cleaners'],
 ['53', ' Married-civ-spouse', ' Handlers-cleaners'],
 ['28', ' Married-civ-spouse', ' Prof-specialty']]

In [0]:
# Na początek stwórz RDD, które będzie zawierać tylko takie osoby, które są rozwiedzione: _c5 zawiera słowo Divorced.
# Wypisz 10 pierwszych osób z tego zestawienia, posortowanych według wieku rosnąco.

In [0]:
rdd3 = rdd2.filter(lambda x: (" Divorced" == x[1]) )
rdd3.takeOrdered(5)

Out[108]: [['18', ' Divorced', ' Other-service'],
 ['19', ' Divorced', ' Craft-repair'],
 ['19', ' Divorced', ' Exec-managerial'],
 ['19', ' Divorced', ' Other-service'],
 ['19', ' Divorced', ' Other-service']]

Teraz dokonaj dalszej analizy. Weź pod uwagę zawód. Oblicz, jakim procentem dla tych wszystkich rozwiedzionych są osoby, które pracują na stanowiskach menedżerskich: właściwością _c6 jest Exec-managerial.

In [0]:
rdd4 = rdd3.filter(lambda x: (" Exec-managerial" == x[2]) )
rdd4.take(2)

Out[109]: [['43', ' Divorced', ' Exec-managerial'],
 ['39', ' Divorced', ' Exec-managerial']]

In [0]:
#no. divorced
divorced = rdd3.count()
# no. of div managers 
divorced_manager = rdd4.count()
wynik = (100*divorced_manager)/divorced 
print(f"procent rozwiedzionych managerów: {wynik:.2f} %")

procent rozwiedzionych managerów: 13.59 %